In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import langchain_core, langchain_ollama

#langchain_core : 0.3.10
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)

langchain_core : 0.3.10
langchain_ollama : 0.2.0


In [2]:
# ResponseError: gemma2:9b does not support tools
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

In [3]:
llm.invoke(input="What is 3 * 12? Also, what is 11 + 49?").content

'To calculate the first expression:\n\n3 * 12 = 36\n\nTo calculate the second expression:\n\n11 + 49 = 60'

In [4]:
from langchain_core.tools import tool

In [6]:
@tool
def add(a:int, b:int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a:int, b:int) -> int:
    """Multiplies a and b."""
    return a * b    

In [7]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

In [9]:
messages = [HumanMessage(query)]

In [10]:
ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.tool_calls)

[{'name': 'multiply', 'args': {'a': '3', 'b': '12'}, 'id': '3ac2535e-9adf-4569-ba29-9032cbe1ae18', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': '11', 'b': '49'}, 'id': '3fdf2317-5fd6-403c-a492-0dfcf8f94fb8', 'type': 'tool_call'}]


In [11]:
messages.append(ai_msg)

In [12]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-14T08:59:50.098898Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': '3', 'b': '12'}}}, {'function': {'name': 'add', 'arguments': {'a': '11', 'b': '49'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 2913785542, 'load_duration': 31428000, 'prompt_eval_count': 231, 'prompt_eval_duration': 915504000, 'eval_count': 44, 'eval_duration': 1964793000}, id='run-fcda94d0-6e41-4fe4-97e5-6d16896fe4aa-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '12'}, 'id': '3ac2535e-9adf-4569-ba29-9032cbe1ae18', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': '11', 'b': '49'}, 'id': '3fdf2317-5fd6-403c-a492-0dfcf8f94fb8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens'

In [13]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply":multiply}[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

In [14]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-14T08:59:50.098898Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': '3', 'b': '12'}}}, {'function': {'name': 'add', 'arguments': {'a': '11', 'b': '49'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 2913785542, 'load_duration': 31428000, 'prompt_eval_count': 231, 'prompt_eval_duration': 915504000, 'eval_count': 44, 'eval_duration': 1964793000}, id='run-fcda94d0-6e41-4fe4-97e5-6d16896fe4aa-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '12'}, 'id': '3ac2535e-9adf-4569-ba29-9032cbe1ae18', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': '11', 'b': '49'}, 'id': '3fdf2317-5fd6-403c-a492-0dfcf8f94fb8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens'

In [15]:
ai_final_msg = llm_with_tools.invoke(messages)

In [16]:
ai_final_msg.content

'The result of multiplying 3 by 12 is: 36\n\nThe result of adding 11 and 49 is: 60'